In [21]:
import csv
import sys
import collections
import os.path
import requests

# on exit clean-ups
import atexit

# cassandra libs
from cassandra.cluster import Cluster
from cassandra.query import dict_factory


# neo4j libs
from py2neo import Graph
from py2neo import Node, Relationship
from py2neo import authenticate

# neo4j graph connector
authenticate("localhost:7474", "neo4j", "1sTep123")
graph = Graph()
# delete entire graph
#graph.delete_all()

# bool flag database connections
cassandraDbOn=False
neo4jDbOn=False

def dbCleanUP(cassandraDbOn,neo4jDbOn):
    if cassandraDbOn:
    	print 'cleaning Cassandra state'
    	session.shutdown();
    	cluster.shutdown();

atexit.register(dbCleanUP,True,True)

def moveContentModel():
    baseURL = "http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/getContent/"
    listURL = "http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/content/list"

    # neo4j graph connector
    graph = Graph()
    
    url = listURL
    resp = requests.get(url).json()
    # no of content
    contentList = resp["result"]["contents"]
    for contentListDict in contentList:
        # check if there is an identifier for this content
        if(not contentListDict.has_key('identifier')):
            continue
    
        # check if there is an identifier for this content
        identifier = contentListDict['identifier']

        # create a node for this Content
        #node=Node("Content",id=identifier)
        #graph.merge(node,"Content","id")
        node = graph.merge_one("Content","id",identifier)
    
        url = baseURL + identifier
        resp = requests.get(url)

        if(resp.status_code!=200):
            continue

        resp =  resp.json()

        concept=None
        learningObjective=None
        
        contentDict = resp["result"]["content"]

        #if(contentDict.has_key('concepts')):
            # this forms a "relationship" in the graph
            #concepts = contentDict['concepts']
            
        if(contentDict.has_key('learningObjective')):
            # this forms a "relationship" in the graph
            learningObjective = contentDict['learningObjective']
            node.properties['learningObjective']=learningObjective
            node.push()    
    
        print('** id',identifier,'added learningObjective:',learningObjective,'**')
    
        # This relationship exists in db      
        #for concept in concepts:
           # print('concept:',concept)
            #node2=Node("Concept",id=concept)
            #graph.merge(node2,"Concept","id")
            #graph.create(Relationship(node2, "COVERED_IN", node))



# move concept map 
def moveConceptMap():
    graph = Graph()
    # delete entire graph

    url="http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/domain/map"
    resp = requests.get(url).json()

    # move all concepts
    conceptList = resp["result"]["concepts"]
    for conceptDict in conceptList:
        identifier=None
        if(not conceptDict.has_key('identifier')):
            continue

        identifier = conceptDict['identifier']
        # create/find node
        #node=Node("Concept",id=identifier)
        #graph.merge(node,"Concept","id")
        node = graph.merge_one("Concept","id",identifier)
        #node = graph.merge_one("Concept","id",identifier)
       
        if(conceptDict.has_key('learningObjective')):
            learningObjective = conceptDict['learningObjective']
            node.properties["learningObjective"]=learningObjective
            node.push()

        if(conceptDict.has_key('misconceptions')):
            misconceptions = conceptDict['misconceptions']
            node.properties["misconceptions"]=misconceptions
            node.push()


moveConceptMap();
moveContentModel();


('** id', u'Test_QAT_46', 'added learningObjective:', None, '**')
('** id', u'Test_QA_13', 'added learningObjective:', None, '**')
('** id', u'numeracy_374', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.numchart.worksheet', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.addobj.worksheet', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.money.worksheet', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.time.worksheet', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.moreless.worksheet', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.math.magic', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.counting.worksheet', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.num.build.sentence', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.lit.moon_cap.story', 'added learningObjective:', None, '**')
('** id', u'org.ekstep.lit.haircut.story', 'added learningObjec

In [18]:
graph = Graph()
contentRec = graph.cypher.execute("MATCH (a:Concept)-[r:COVERED_IN]->(b:Content) RETURN b.id AS Content,b.learningObjective as LearningObjective, b.misconceptions as MisconceptionCovered")
print('Recomended content:')
contentRec

Recomended content:


    | Content                       | LearningObjective | MisconceptionCovered
----+-------------------------------+-------------------+----------------------
  1 | domain_54564                  |                   |                     
  2 | domain_14625                  |                   |                     
  3 | domain_13483                  |                   |                     
  4 | domain_14589                  |                   |                     
  5 | domain_14302                  |                   |                     
  6 | domain_13313                  |                   |                     
  7 | domain_44234                  |                   |                     
  8 | domain_48661                  |                   |                     
  9 | domain_45479                  |                   |                     
 10 | domain_48225                  |                   |                     
 11 | domain_48659                  |              